In [80]:
import pandas as pd 
import plotly.express as px
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';')
dfo['timestamp']=pd.to_datetime(dfo['timestamp'])
#df_day=dfo.copy()
#df_day['h']=df_day['timestamp'].dt.hour

In [81]:
#px.scatter(dfo,x='timestamp',y='DeviationID ValueY',color='DeviationID ValueY',marginal_x="rug")

In [82]:
df=dfo.copy()
df=df.drop(columns=["timestamp","CuStepNo ValueY"])
c=list(df.columns)
c[0]="Dev"
df.columns=c
df=df[df['Dev']!=0]
df['Dev']=df['Dev']!=1
df['Dev']=df['Dev'].astype('int')
df_clean=df.copy()

In [83]:
df_clean=df_clean.sample(250000)
df_clean.to_csv('..\\Data\\data_01_clean.csv')

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, recall_score, precision_score, accuracy_score

In [85]:
X_train,X_test,y_train,y_test=train_test_split(df_clean.drop(columns=['Dev']),df_clean['Dev'],shuffle=False,random_state=42)

In [86]:
model=RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

RandomForestClassifier()

In [87]:
y_pred=model.predict(X_train)
print(f'R2:{r2_score(y_train,y_pred)}\nAccuracy: {accuracy_score(y_train,y_pred)}\nRecall: {recall_score(y_train,y_pred)}\nPrecision: {precision_score(y_train,y_pred)}')

R2:1.0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0


In [88]:
y_pred=model.predict(X_train)
tn, fp, fn, tp = confusion_matrix(y_train,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 25093
FP: 0
FN: 0
TP: 162407


In [89]:
y_pred=model.predict(X_test)
print(f'R2:{r2_score(y_test,y_pred)}\nAccuracy: {accuracy_score(y_test,y_pred)}\nRecall: {recall_score(y_test,y_pred)}\nPrecision: {precision_score(y_test,y_pred)}')


R2:0.8438243706540643
Accuracy: 0.982016
Recall: 0.9913285732735558
Precision: 0.9879746253562564


In [90]:
y_pred=model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 7645
FP: 654
FN: 470
TP: 53731


In [91]:

#from sklearn.externals import joblib
from joblib import dump, load
dump(model, 'rf.pkl')


['rf.pkl']

In [92]:
from sklearn.cluster import DBSCAN
import numpy as np
clustering = DBSCAN(eps=3, min_samples=2).fit(X_train)



In [93]:
from sklearn.ensemble import IsolationForest
from joblib import dump, load

ISF = IsolationForest(n_estimators=100)
ISF.fit(X_train) 
dump(ISF, 'ISF.pkl')


['ISF.pkl']

In [94]:
from sklearn.neighbors import LocalOutlierFactor
from joblib import dump, load

lof = LocalOutlierFactor(n_neighbors=20,novelty=True)
lof.fit(X_train) 
dump(lof, 'lof.pkl')

['lof.pkl']